In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)
tf.reset_default_graph()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
n_input = 28
max_time = 28
#隐层个数
lstm_size = 100
n_class = 10
batch_size = 50
n_batch_size = mnist.train.num_examples // batch_size

In [3]:
x = tf.placeholder(tf.float32,[None,28*28])
y = tf.placeholder(tf.float32,[None,10])

In [4]:
weight = tf.Variable(tf.truncated_normal([lstm_size,n_class],stddev = 0.1))
biases = tf.Variable(tf.constant(0.1,shape=[n_class]))

In [5]:
def rnn(X,weights,biases):
    input_ = tf.reshape(X,[-1,max_time,n_input])
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(lstm_size)

    outputs,final_state = tf.nn.dynamic_rnn(lstm_cell,input_,dtype=tf.float32)
    
    results = tf.nn.softmax(tf.matmul(final_state[1],weights)+biases,1)
    return results

In [6]:

prediction = rnn(x,weight,biases)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))


In [7]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(6):
        for batch in range(n_batch_size):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step, feed_dict={x: batch_xs,y: batch_ys})
            if(batch%100==0):
                print(str(batch)+"/" + str(n_batch_size))
        acc = sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels})
        print("Iter" + str(epoch) + " ,Testing Accuracy = " + str(acc))

0/1100
100/1100
200/1100
300/1100
400/1100
500/1100
600/1100
700/1100
800/1100
900/1100
1000/1100
Iter0 ,Testing Accuracy = 0.7521
0/1100
100/1100
200/1100
300/1100
400/1100
500/1100
600/1100
700/1100
800/1100
900/1100
1000/1100
Iter1 ,Testing Accuracy = 0.8091
0/1100
100/1100
200/1100
300/1100
400/1100
500/1100
600/1100
700/1100
800/1100
900/1100
1000/1100
Iter2 ,Testing Accuracy = 0.8918
0/1100
100/1100
200/1100
300/1100
400/1100
500/1100
600/1100
700/1100
800/1100
900/1100
1000/1100
Iter3 ,Testing Accuracy = 0.9078
0/1100
100/1100
200/1100
300/1100
400/1100
500/1100
600/1100
700/1100
800/1100
900/1100
1000/1100
Iter4 ,Testing Accuracy = 0.9236
0/1100
100/1100
200/1100
300/1100
400/1100
500/1100
600/1100
700/1100
800/1100
900/1100
1000/1100
Iter5 ,Testing Accuracy = 0.9239
